CONNECTIONS - Produce data on the number of connections between each zone within 5KM radius, 10KM radius, 20KM radius etc.

In [1]:
"""
Library code
"""
import numpy as np
import struct
import time

#where the output file is written to
outputdir = 'outputs/'

"""
Load QUANT matrix, but do it fast
"""
def loadQUANTMatrixFAST(filename):
    start_time = time.process_time()
    with open(filename,'rb') as f:
        (m,) = struct.unpack('i', f.read(4))
        (n,) = struct.unpack('i', f.read(4))
        print("loadQUANTMatrixFAST::m=",m,"n=",n)
        matrix = np.arange(m*n,dtype=np.float64).reshape(m, n) #and hopefully m===n, but I'm not relying on it
        for i in range(0,m):
            data = struct.unpack('{0}f'.format(n), f.read(4*n)) #read a row back from the stream - data=list of floats
            #for j in range(0,n):
            #    matrix[i,j] = data[j]
            matrix[i,:] = data
        end_time = time.process_time()
        print("loadQUANTMatrixFAST:: ",str(end_time-start_time),"secs")
        return matrix
################################################################################
    
"""
countScenarios
Count how many one link scenarios there are for a given radius for each origin zone i.
NOTE: there is a similar function in PyQUANT debug.py called debug_countScenarios which uses the scenario generator.
This should be functionally identical, but self-contained. However, this version returns data as a numpy array (zonei,count).
"""
def countOneLinkScenarios(radiusKM,N,Lij_mode):      
    result = np.zeros(N)
    for i in range(0,N):
        count=0
        for j in range(0,N):
            if Lij_mode[i,j]<=radiusKM:
                count+=1
        #end for
        result[i]=count
    #end for
    return result
################################################################################

In [2]:
#produce lists of number of connections between zones
#input data - zone codes file containing zone locations and a distance matrix for each mode
import os
import numpy as np
import pandas as pd

ModelRunsDir = 'C:\\richard\\github\\PyQUANT3\\inputs\\model-runs' #location of data
DisCrowflyVertexRoadsKMFilename = 'dis_crowfly_vertex_roads_KM.bin' #NOTE: these can come from OSF!
DisCrowflyVertexBusKMFilename = 'dis_crowfly_vertex_bus_KM.bin'
DisCrowflyVertexGBRailKMFilename = 'dis_crowfly_vertex_gbrail_KM.bin'
ZoneCodesFilename = 'EWS_ZoneCodes.xml'

Lij_road = loadQUANTMatrixFAST(os.path.join(ModelRunsDir,DisCrowflyVertexRoadsKMFilename))
Lij_bus = loadQUANTMatrixFAST(os.path.join(ModelRunsDir,DisCrowflyVertexBusKMFilename))
Lij_rail = loadQUANTMatrixFAST(os.path.join(ModelRunsDir,DisCrowflyVertexGBRailKMFilename))

#load zone codes xml file into a pandas dataframe
with open(os.path.join(ModelRunsDir,ZoneCodesFilename)) as f:
    df_ZoneCodes = pd.read_xml(f)
    print(df_ZoneCodes.head(10))
    print("ZoneCodes has "+str(len(df_ZoneCodes.index))+" rows")

N = len(df_ZoneCodes.index)
df_ZoneCodes.set_index('zonei') #make sure we're adding the numpy arrays from countOneLinkScenarios in the zonei order

#now count up the number of scenarios for each radius and mode, along with the number of zones that are disconnected (out links<2)
for radius in range(1,31):
    count_road=countOneLinkScenarios(radius,N,Lij_road)
    count_road_less2 = (count_road < 2).sum()
    df_ZoneCodes['road'+str(radius)] = count_road
    count_bus=countOneLinkScenarios(radius,N,Lij_bus)
    count_bus_less2 = (count_bus < 2).sum()
    df_ZoneCodes['bus'+str(radius)] = count_bus
    count_rail=countOneLinkScenarios(radius,N,Lij_rail)
    count_rail_less2 = (count_rail < 2).sum()
    df_ZoneCodes['rail'+str(radius)] = count_rail
    #todo: you could write this to a file too!
    print(radius,'KM: ',np.sum(count_road), np.sum(count_bus), np.sum(count_rail), count_road_less2, count_bus_less2, count_rail_less2 )
df_ZoneCodes.to_csv(outputdir+'ZoneCodes_radiusKM.csv')


loadQUANTMatrixFAST::m= 8436 n= 8436
loadQUANTMatrixFAST::  3.4375 secs
loadQUANTMatrixFAST::m= 8436 n= 8436
loadQUANTMatrixFAST::  3.4375 secs
loadQUANTMatrixFAST::m= 8436 n= 8436
loadQUANTMatrixFAST::  3.5 secs
     areakey  zonei        lat       lon  osgb36_east  osgb36_north       area
0  E02000001      0  51.515000 -0.090516    532482.75     181269.56  2897837.5
1  E02000002      1  51.588420  0.141110    548312.70     189878.02  2161565.0
2  E02000003      2  51.575100  0.142558    548456.44     188399.88  2141516.0
3  E02000004      3  51.555622  0.178484    551010.00     186307.53  2492948.0
4  E02000005      4  51.561592  0.144949    548666.10     186902.60  1187953.0
5  E02000007      5  51.558270  0.159004    549651.06     186562.12  1735997.9
6  E02000008      6  51.552820  0.136958    548140.90     185910.88  1419999.0
7  E02000009      7  51.551743  0.119994    546968.44     185757.10   960733.6
8  E02000010      8  51.547050  0.150028    549065.60     185296.10  1121450